In [1]:
#!pip install textattack

In [2]:
import numpy as np
import pandas as pd

from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

In [3]:
!gpustat

seven                Wed Mar 10 13:36:01 2021  440.64
[0] TITAN RTX        | 32'C,   0 % |    11 / 24220 MB |
[1] TITAN RTX        | 69'C,  37 % |  2662 / 24220 MB | cwb(2651M)
[2] TITAN RTX        | 33'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 87'C,  99 % |  9928 / 24220 MB | cwb(3411M) cwb(3063M) cwb(3443M)


In [4]:
device = 'cuda:0'

In [5]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [6]:
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
def load_dataset_from_csv(filename):
    df = pd.read_csv(filename).sample(frac=1, random_state=42)
    #df = df[df.target != 'A22']
    unique_labels = np.unique(sorted(df.target.values))
    label_to_id = {label: label_id for label_id, label in enumerate(unique_labels)}
    return list(df.text.values), [label_to_id[label] for label in df.target.values]

In [8]:
train_texts, train_labels = load_dataset_from_csv('/home/mlepekhin/data/en_train')
test_texts, test_labels = load_dataset_from_csv('/home/mlepekhin/data/en_test')

In [9]:
import pickle

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
#train_encodings = pickle.load(open('en_train_xlm_roberta_encodings.pcl', 'rb'))
#test_encodings = pickle.load(open('en_test_xlm_roberta_encodings.pcl', 'rb'))

In [10]:
train_dataset = GenreDataset(train_encodings, train_labels)
test_dataset = GenreDataset(test_encodings, test_labels)

In [11]:
from torch.utils.data import DataLoader

#train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12)

In [12]:
!mkdir xlm_roberta_en

mkdir: cannot create directory ‘xlm_roberta_en’: File exists


In [13]:
from sklearn.metrics import accuracy_score
from transformers import AdamW
from tqdm import tqdm
from collections import defaultdict

In [14]:
from matplotlib import pyplot as plt

In [15]:
for val_step in range(5):
    model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=11).to(device)
    optim = AdamW(model.parameters(), lr=5e-5)
    total_test_accuracy = 0.0
    
    train_subset = defaultdict(list)
    train_subset_labels = []
    val_subset = defaultdict(list)
    val_labels = []
    
    for i in range(len(train_dataset)):
        if i >= val_step * 0.2 * len(train_dataset) and i < (val_step+1) * 0.2 * len(train_dataset):
            val_subset['input_ids'].append(train_encodings['input_ids'][i])
            val_subset['attention_mask'].append(train_encodings['attention_mask'][i])
            val_labels.append(train_labels[i])
        else:
            train_subset['input_ids'].append(train_encodings['input_ids'][i])
            train_subset['attention_mask'].append(train_encodings['attention_mask'][i])
            train_subset_labels.append(train_labels[i])
    
    train_loader = DataLoader(GenreDataset(train_subset, train_subset_labels), batch_size=12, shuffle=True)
    val_loader = DataLoader(GenreDataset(val_subset, val_labels), batch_size=12)

    for epoch in range(7):
        model.train()
        train_accuracy = 0
        item_count = 0

        for batch in tqdm(train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            item_count += batch['labels'].shape[0]
            #start_positions = batch['start_positions'].to(device)
            #end_positions = batch['end_positions'].to(device)
            output = model(input_ids, attention_mask=attention_mask, labels=labels)
            cur_accuracy = np.sum(batch['labels'].numpy() == np.argmax(output.logits.cpu().detach().numpy(), axis=1))
            train_accuracy += cur_accuracy 
            loss = output.loss
            loss.backward()
            optim.step()
            print(f'loss: {loss.item()}, accuracy: {train_accuracy / item_count}')
            
        model.eval()
        all_outputs = [] 

        for batch in tqdm(val_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            all_outputs.append(np.argmax(outputs.cpu().detach().numpy(), axis=1))
        all_outputs = np.concatenate(all_outputs)
        val_accuracy = accuracy_score(all_outputs, val_labels)
        print(f"epoch: {epoch} val accuracy: {val_accuracy}")
        
        model.eval()
        all_outputs = [] 

        for batch in tqdm(test_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            all_outputs.append(np.argmax(outputs.cpu().detach().numpy(), axis=1))
        all_outputs = np.concatenate(all_outputs)
        test_accuracy = accuracy_score(all_outputs, test_labels)

        print(f"epoch: {epoch} test accuracy: {test_accuracy}")
        if test_accuracy > total_test_accuracy:
            total_test_accuracy = test_accuracy
            torch.save(model.state_dict(), f'xlm_roberta_en/state_dict_val_{val_step}.pt')


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

loss: 2.4308598041534424, accuracy: 0.16666666666666666


  2%|▏         | 2/85 [00:00<00:36,  2.25it/s]

loss: 2.4700610637664795, accuracy: 0.08333333333333333


  4%|▎         | 3/85 [00:01<00:35,  2.31it/s]

...

loss: 0.14042364060878754, accuracy: 0.9126984126984127
loss: 0.22894345223903656, accuracy: 0.9130434782608695


  3%|▎         | 1/36 [00:00<00:04,  7.39it/s]

epoch: 6 val accuracy: 0.6785714285714286


100%|██████████| 36/36 [00:04<00:00,  7.49it/s]


epoch: 6 test accuracy: 0.6919431279620853


In [16]:
!gpustat

seven                Wed Mar 10 14:06:26 2021  440.64
[0] TITAN RTX        | 31'C,   0 % |    11 / 24220 MB |
[1] TITAN RTX        | 66'C,  31 % |  2662 / 24220 MB | cwb(2651M)
[2] TITAN RTX        | 68'C,   0 % | 20738 / 24220 MB | mlepekhin(20727M)
[3] TITAN RTX        | 86'C,  98 % |  9928 / 24220 MB | cwb(3411M) cwb(3063M) cwb(3443M)
